In [ ]:
import json
from typing import List, Dict

import requests
from together import Together
from transformers import AutoTokenizer

# Предобработка входных данных

В данном задании мы будем ходить в онлайн модель. Предлагается все также ходить в together.ai, т.к. они дают $5 кредита при регистрации.

Вначале давайте руками поиграемся с API, посмотрим, как походы в API соотносятся с тем, что мы делали в домашнем задании "Доступные LLM"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3.1-70B")

## Ручное форматирование промпта - 5 баллов

Давайте попробуем собрать вход для llama3.1 руками, для этого допишем функцию `format_messages_to_prompt`.
Она принимает messages - массив словарей, где указаны роли и текст сообщений, а возвращает она текст в формате, который нужно подать модели.

Например для истории сообщений

```python
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]
```

должен выдаваться итоговый промпт

```text
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>
```

Что важно:
1. Текст начинается со спецтокена bos
2. Дальше идет заголовок start_header_id + end_header_id, которые содержат роль
3. Дальше после \n\n идет текст, заканчивающийся на eot_id
4. Дальше следующий заголовок с новой ролью и т.д.

**Важно** - в данной функции нельзя использовать `tokenizer.apply_chat_template`

In [ ]:
def format_messages_to_prompt(messages: List[Dict[str, str]]) -> str:
    # Ваш код здесь
    return ""


messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]



reference_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>"""


assert format_messages_to_prompt(messages) == reference_text

Мы также помним, что раньше у нас была `tokenizer.apply_chat_template`. Т.к. у нас неофициальный форк llama3.1, то chat_template в токенайзер нам не завезли, поэтому придется добавить его руками

In [ ]:
chat_template = """
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- "Today Date: " + date_string + "\n\n" }}
{%- if tools is not none and not tools_in_user_message %}
    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
{%- endif %}
{{- system_message }}
{{- "<|eot_id|>" }}

{#- Custom tools are passed in a user message with some extra guidance #}
{%- if tools_in_user_message and not tools is none %}
    {#- Extract the first user message so we can plug it in here #}
    {%- if messages | length != 0 %}
        {%- set first_user_message = messages[0]['content']|trim %}
        {%- set messages = messages[1:] %}
    {%- else %}
        {{- raise_exception("Cannot put tools in the first user message when there's no first user message!") }}
{%- endif %}
    {{- '<|start_header_id|>user<|end_header_id|>\n\n' -}}
    {{- "Given the following functions, please respond with a JSON for a function call " }}
    {{- "with its proper arguments that best answers the given prompt.\n\n" }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
    {{- first_user_message + "<|eot_id|>"}}
{%- endif %}

{%- for message in messages %}
    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}
        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' }}
    {%- elif 'tool_calls' in message %}
        {%- if not message.tool_calls|length == 1 %}
            {{- raise_exception("This model only supports single tool-calls at once!") }}
        {%- endif %}
        {%- set tool_call = message.tool_calls[0].function %}
        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- "<|python_tag|>" + tool_call.name + ".call(" }}
            {%- for arg_name, arg_val in tool_call.arguments | items %}
                {{- arg_name + '="' + arg_val + '"' }}
                {%- if not loop.last %}
                    {{- ", " }}
                {%- endif %}
                {%- endfor %}
            {{- ")" }}
        {%- else  %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- '{"name": "' + tool_call.name + '", ' }}
            {{- '"parameters": ' }}
            {{- tool_call.arguments | tojson }}
            {{- "}" }}
        {%- endif %}
        {%- if builtin_tools is defined %}
            {#- This means we're in ipython mode #}
            {{- "<|eom_id|>" }}
        {%- else %}
            {{- "<|eot_id|>" }}
        {%- endif %}
    {%- elif message.role == "tool" or message.role == "ipython" %}
        {{- "<|start_header_id|>ipython<|end_header_id|>\n\n" }}
        {%- if message.content is mapping or message.content is iterable %}
            {{- message.content | tojson }}
        {%- else %}
            {{- message.content }}
        {%- endif %}
        {{- "<|eot_id|>" }}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}
""".strip()
tokenizer.chat_template = chat_template

## Автоматическая сборка промпта - 5 баллов

Давайте вспомним теперь на деле, как используется chat_template! Попробуем использовать функцию `tokenizer.apply_chat_template`

In [ ]:
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]

prompt = tokenizer.apply_chat_template ...# Ваш код здесь

In [ ]:
reference_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>"""

assert prompt == reference_prompt

Обратите внимание, что в заданном chat_template указаны Cutting Knowledge Date, т.е. до данные до какого периода видела модели, и Today Date - захардкоженная дата текущего диалога.

**Вопрос, обязательно напишите свой ответ здесь!**
На что влияет аргумент `add_generation_prompt` в функции `tokenizer.apply_chat_template`? Зачем его использовать?

## Походы в API - 10 баллов

Теперь давайте посмотрим, как можно ходить в API. Для примера мы будем ходить в together.ai, который щедро предоставляет $5 всем зарегистрировавшимся. Вообще говоря различных провайдеров много, API у них у всех очень похожий, т.к. все мимикрируют под OpenAI.

In [ ]:
# Вставьте свой ключ из https://api.together.ai/
API_KEY = "PASTE YOUR KEY HERE"

Есть несколько способов сходить в API. Можно ходить напрямую через библиотеку **requests**. Допишите post запрос в `url` с данными `data` и заголовками `headers`. 

In [ ]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "messages": messages
}

response = requests ...# Ваш код здесь
model_answer = response.json() ...# Ваш код здесь
assert "london" in model_answer.lower()

Мы подали messages, дальше они каким-то образом собрались в promt и подались модели. Мы не знаем, какой промпт используется на стороне провайдера. Вспомним про Today Date из предыдущего пункта задания - использует ли его together? Обновляют ли они его сегодняшним днем или оставляют 26 июля? Если обновляют, то по какому часовому поясу?

Чтобы ответы на эти и многие другие вопросы не мучали нас по ночам, можно использовать prompt формат, а именно подать модели текст напрямую на генерацию. Давайте для этого используем `tokenizer.apply_chat_template`. Модель будет принимать текст ровно так, как вы его подадите, без каких-либо предобработок. Подумайте, нужно ли вам использовать аргумент `add_generation_prompt`?

Чтобы послать запрос напрямую, нужно в предыдущем запросе убрать messages, который представляет из себя список словарей, и послать поле prompt - строку с промптом для модели.

In [ ]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

prompt = tokenizer.apply_chat_template ...

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    ...# Ваш код здесь
}

response = requests ...# Ваш код здесь
model_answer = response.json() ...# Ваш код здесь
assert "london" in model_answer.lower() and "assistant" not in model_answer.lower()

## Клиент - 5 баллов

Теперь мы понимаем общую схему взаимодействия с провайдером - они предоставляют апи, куда можно посылать или промтп или историю диалога. При посылке промпта вся ответственность за формат ложится на нас, при посылке messages форматтинг происходит на стороне провайдера, но мы не всегда представляем, как он работает. Выбор в пользу того или иного варианта всегда остается на вас.

Мы использовали выше библиотеку requests, чтобы послать HTTP-запрос на сервера together, однако есть способ и проще - python client. Давайте познакомимся с ним поближе. Для этого давайте используем функцию `client.chat.completions.create`. Также давайте добавим опции сэмплинга, которые в этой функции поддержаны. Их можно посылать и в запросах через requests, но мы здесь и далее будем пользоваться клиентом.
* top_k = 100
* temperature = 0.5
* top_p = 0.9
* repetition_penalty = 1.05

In [ ]:
client = Together(api_key=API_KEY)

In [ ]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

response = client.chat.completions.create(
    ...# Ваш код здесь
)

response_text = response ...# Ваш код здесь
assert "london" in response_text.lower()

Аналогично посылать просто prompt можно через `client.completions.create`.

## Tools - 5 баллов

Давайте теперь посмотрим, как можно использовать tools в связке с моделями. У нас есть функция, которая входит в базу данных и получает информацию о юзере. Базы данных, конечно же, у нас никакой нет, но у нас есть некоторая функция, которая эмулирует это поведение, так что давайте попробуем ее описать.


In [ ]:
def get_user_info_from_db(person_name: str) -> Dict[str, str]:
    database = {
        "ilya": {
            "job": "Software Developer",
            "pets": "dog",
        },
        "farruh": {
            "job": "Senior Data & Solution Architect",
            "hobby": "travelling, hiking",
        },
        "timur": {
            "job": "DeepSchool Founder",
            "city": "Novosibirsk",
        }
    }
    no_info = {"err": f"No info about {person_name}"}
    return database.get(person_name.lower(), no_info)

print(get_user_info_from_db("Timur"))

Давайте попробуем описать эту функцию в формате json, чтобы модель могла ее увидеть!
Заполните поля в определении дальше

In [ ]:
get_user_info_from_db_tool = {
    "type": "function",
    "function": {
        "name": "get_user_info_from_db",
        "description": ..., # Напишите, что функция делает своими словами
        "parameters": {
            "type": "object",
            "properties": {
                "person_name": {
                    "type": "string",
                    "description": ...# Опишите смысл аргумента
                }
            },
            "required": [...] # укажите обязательные аргументы для функции
        }
    }
}

Теперь давайте подадим это описание в `tokenizer.apply_chat_template`. Обратите внимание на его аргумент `tools`! Не забудьте `add_generation_prompt`, если он нужен.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
prompt = tokenizer.apply_chat_template ...# Ваш код здесь
print(prompt)

Давайте пошлем наш запрос в модель. На выбор 2 модели, если не будет работать с 8b, то предлагается посылать в 70b.
Для данного запроса для 8b был подобран работающий `seed=9706540181089681000`, который можно подать в функцию.

Давайте воспользуемся `client.completions.create` для генерации ответа от модели.

In [ ]:
model_8b = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
model_70b = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

In [ ]:
response_8b = client.completions.create ... # Ваш код здесь
response_70b = client.completions.create... # Ваш код здесь

print(response_8b.choices[0].text)
print(response_70b.choices[0].text)

Если все хорошо, то мы получили ответ от модели, который выглядит как некоторый структурированный вывод, который можно использовать для вызова модели. Давайте попробуем написать функцию, которая принимает ответ модели в "сыром виде", выбирает, какую функцию с какими аргументами вызвать. 

Здесь нам поможет FUNCTION_REGISTRY и то, что параметры в функцию можно передавать как словарь, например так
```python
def foo(a, b, c):
    print(a, b, c)

obj = {'b':10, 'c':'lee'}

foo(100, **obj)
```

In [ ]:
FUNCTION_REGISTRY = {"get_user_info_from_db": get_user_info_from_db}
# На случай, если модель не генерит function call
reference_answer = """{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}"""


def parse_function_call(model_answer):
    # Ваш код здесь
    # 1. Проверим, является ли это function call.
    # 2. Вызов нужной функции с указанными аргументами
    ...
    

assert parse_function_call(reference_answer) == get_user_info_from_db("Ilya")

Теперь давайте попробуем объединить все это в историю диалога и сгенерировать моделью финальный ответ.
Для этого в messages, где хранится наша история диалога нужно добавить
1. Вызов function call моделью с ролью ХХХ (это часть задания, напишите сами)
2. Ответ function call с ролью tool

После этого данный промпт нужно послать модели снова, чтобы получить финальный ответ.
Для этого опять используем `tokenizer.apply_chat_template` и `client.completions.create`.

В зависимости от модели может понадобиться убрать tools (на 8b, 70b должна справиться). Для 8b опять же подобран seed=2017684582943914000

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
# Добавляем ответ модели
messages.append(...)# Ваш код здесь
# Добавляем ответ tool
messages.append(...)# Ваш код здесь

prompt = tokenizer.apply_chat_template ... # Ваш код здесь
# print(prompt)

response_8b = client.completions.create(model=model_70b, prompt=prompt, seed=2017684582943914000)
print(response_8b.choices[0].text)

Теперь давайте посмотрим на chat-API, как обрабатываются function calls там?
Используем для этого уже знакомый `client.chat.completions.create`, обратим внимание на аргумент tools внутри него. Здесь рекомендуется использовать 70b модель. На всякий случай работающий seed=14157400267283583000

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = client.chat.completions.create ... # Ваш код здесь

Мы можем видеть, что у нас не работает предыдущий подход с полем `content`, однако должно было появиться поле `tool_calls`, которое содержит в себе информацию о вызове инструмента

In [ ]:
response.choices[0].message.tool_calls

# Использование библиотек

Теперь, когда мы руками прошли весь пути обработки function call можно посмотреть уже на готовые инструменты.
Мы много чего сделали руками:
1. Писали описание функции
2. Обрабатывали ответ
3. Вызывали функцию
4. Возвращали все это в модель

Давайте теперь посмотрим, как оно работает в библиотеках!

**NB** - библиотеки развиваются и вполне, возможно, что к концу курса те интерфейсы, которые мы используем в этом домашнем задании будут уже неактуальны, но я уверен, что знаний и принципов, полученных из этих заданий хватит, чтобы адаптироваться к будущим вызовам!

In [ ]:
# ! pip install langchain==0.2.16 llama-index-core==v0.11.16 langchain-together==0.2.0 llama-index-llms-together==0.2.0

# LangChain - 5 баллов

In [ ]:
import os
from langchain_together import ChatTogether
from langchain_core.tools import tool

Давайте ознакомимся с langchain-интеграцией together.ai

In [ ]:
os.environ["TOGETHER_API_KEY"] = API_KEY


llm = ChatTogether(...)# Ваш код здесь

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = llm.invoke(messages)
print(response.content)

Теперь, когда мы разобрались, как базово работать с langchain, давайте попробуем добавить инструментов. Чтобы нам было не так скучно, давайте напишем новую функцию, которая считает "волшебную операцию".

Эта функция принимает 2 строки, возвращает строку строку b в обратном порядке, сконкатенированную со строкой a. Допишите эту функцию.

In [ ]:
def magic_operation(a, b):
    ... # Ваш код здесь

assert magic_operation("456", "321") == "123456"

Теперь давайте обернем эту функцию в декоратор tool из langchain, аннотируем типы и допишем docstring. После этого можно будет автоматически сгенерировать описани функции в function call формате!

In [ ]:
... # декоратор
def magic_operation_tool(a, b): # аннотации типов
    """""" # docstring
    return magic_operation(a, b)

print(magic_operation_tool.args_schema.schema())

Теперь давайте попробуем подать запрос в нашу LLM и обогатить ее нашим function_call. Для этого нужна функция `llm.bind_tools`.

In [ ]:
llm_with_tools = llm.bind_tools...# Ваш код здесь

Теперь давайте как и раньше:
1. Сгенерируем ответ на messages
2. Проверим в ответе resp.tool_calls, вызовем нужный инструмент
3. Расширим messages ответом модели и ответом инструмента, сгенерируем финальный ответ.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"}
]
resp = llm_with_tools.invoke(messages)
...# Ваш код здесь

In [ ]:
assert len(messages) == 4

In [ ]:
res = llm.invoke(messages).content
assert "123456" in res

# LlamaIndex - 5 баллов

Аналогичный инструмент LlamaIndex. В ней не так хороша поддержка function calls не для OpenAI, поэтому придется забежать вперед и использовать ReActAgent.

In [ ]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

In [ ]:
llm = TogetherLLM(model=model_70b, api_key=API_KEY)

Скопируйте magic_operation_tool из части с langchain сюда,  но без декоратора.

In [ ]:
def magic_operation_tool(a, b): # аннотации типов
    """""" # docstring
    print("INSIDE FUNCTION CALL")
    return magic_operation(a, b)

Мы можем аналогично создать инструмент с помощью `FunctionTool.from_defaults`

In [ ]:
magic_operation_tool_llamaindex = ...# Ваш код здесь
print(magic_operation_tool_llamaindex.metadata)

Давайте создадим ReActAgent: ему нужно передать tools, llm, memory=None и verbose=True

In [ ]:
agent = ReActAgent...# Ваш код здесь

In [ ]:
text = "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"
agent.chat(text)

# Agents - 10

Настала пора сделать своего агента!
Попробуем сделать финансового аналитика. Требования следующие:
бот должен по запросу данных о какой-либо компании смотреть самые большие изменения цены ее акций за последний месяц, после чего бот должен объяснить, с какой новостью это связано.

Предлагается не строить сложную систему с классификаторами, а отдать всю сложную работу агенту. Давайте посмотрим, какие API нам доступны.

Первым делом получение котировок - для этого нам поможет библиотека yfinance. По названию компании и периоду отчетности можно посмотреть открывающие цены на момент открытия и закрытия биржи.

In [ ]:
import yfinance as yf

stock = yf.Ticker("AAPL") # посмотрим котировки APPLE
df = stock.history(period="1mo")
df[["Open", "Close"]]

Для поиска новостей нам поможет https://newsapi.org/
Можно легко получить свой ключ за короткую регистрацию, дается 1000 запросов в день, каждый запрос может включать в себя ключевое слово и промежуток дат. По бесплатному апи ключу дается ровно 1 месяц, что нам подходит.

In [ ]:
api_key = "..." # ваш API ключ здесь!
api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&from={date_from}"

articles = requests.get(api_template.format(keyword="Apple", api_key=api_key, date_from="2025-01-25")).json()

for article in articles["articles"]:
    if article["title"] != "[Removed]":
        print(article["title"])
        print(article["description"])
        break

Очень много статей заблокированы и имеют название `[Removed]`, нужно их отфильтровать. В оставшихся статьях будем брать только title (заголовок) и description (описание или краткий пересказ).

Вам необходимо реализовать [ReAct Agent](https://react-lm.github.io/). Особенность этого агента заключается в том, что он вначале формирует мысль, а потом вызывает действие (function call) для достижения какой-либо цели.

Что нужно сделать:
1. Описать и реализовать function call для определения, в какой день была самая большая разница в цене акций в момент открытия и закрытия биржи. Функция получает один аргумент - название акций компании (например AAPL для Apple), а выдает словарь с 2мя полями: с датой максимальной разницы в ценах и самой разницей в ценах.
2. Описать и реализовать function call для получения 5 релевантных новостей о компании. В качестве аргумента принимаются название компании и дата. Ваша задача - сходить в newsapi, получить новости и вернуть 5 случайных новостей, которые произошли не позже чем день торгов. Если новостей меньше 5, то верните столько, сколько получится.
3. После этого агент должен вернуть ответ, в котором постарается аргументировать изменения в цене.


Реализовывать агента можно любым удобным способом, в том числе взять готовые имплементации.
1. [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/) - вдобавок можно посмотреть предыдущее задание, где он уже используется.
2. [Langchain/Langgraph](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/#code)
3. Написать полностью свою реализацию


Не забудьте, что очень важно описать задачу в промпте: нужно сказать, какие цели у агента и что он должен сделать. У функций должны быть говорящие описания, чтобы LLM без лишних проблем поняла, какие есть функции и когда их использовать. По всем вопросам можно обращаться в наш телеграм-чат в канал "Tools & Agents".
